In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_final_demo = pd.read_csv('../1_data/df_final_demo.txt')
df_final_experiment_clients = pd.read_csv('../1_data/df_final_experiment_clients.txt')
df_final_web_data_pt_1 = pd.read_csv('../1_data/df_final_web_data_pt_1.txt')
df_final_web_data_pt_2 = pd.read_csv('../1_data/df_final_web_data_pt_2.txt')

# Merge web data
df_web = pd.concat([df_final_web_data_pt_1, df_final_web_data_pt_2], ignore_index=True)
df_web["process_step"] = df_web["process_step"].replace('confirm', 'step_4')
df_web["process_step"] = df_web["process_step"].replace('start', 'step_0')
df_web["date_time"] = pd.to_datetime(df_web["date_time"])
df_time = df_web.sort_values(by=['visit_id', 'process_step', 'date_time'])
print(df_time)

#put the name of the steps you wamt to include
steps_required = {'step_0', 'step_1', 'step_2', 'step_3', 'step_4'}
visit_valid = df_time.groupby('visit_id')['process_step'].apply(set).apply(lambda x: steps_required.issubset(x))
df_filtered = df_time[df_time['visit_id'].isin(visit_valid[visit_valid].index)]
print(df_filtered.head())
df_ps_time = pd.DataFrame()
df_ps_time['visit_id'] = df_filtered['visit_id']
df_ps_time['process_step'] = df_filtered['process_step']
df_ps_time['diff_time'] = df_filtered.groupby('visit_id')['date_time'].diff()

#removing all null and negative values
df_ps_time = df_ps_time[df_ps_time['diff_time'] >= pd.Timedelta(0)]
print(df_ps_time.head(50))

        client_id             visitor_id                      visit_id  \
108614    3561384   451664975_1722933822  100012776_37918976071_457913   
108613    3561384   451664975_1722933822  100012776_37918976071_457913   
447951    9056452  306992881_89423906595     1000165_4190026492_760066   
447950    9056452  306992881_89423906595     1000165_4190026492_760066   
447949    9056452  306992881_89423906595     1000165_4190026492_760066   
...           ...                    ...                           ...   
539055    7149380  483112224_46340533900  999992932_41666455053_671149   
539054    7149380  483112224_46340533900  999992932_41666455053_671149   
539072    7149380  483112224_46340533900  999992932_41666455053_671149   
539071    7149380  483112224_46340533900  999992932_41666455053_671149   
539070    7149380  483112224_46340533900  999992932_41666455053_671149   

       process_step           date_time  
108614       step_4 2017-04-26 13:22:17  
108613       step_4 2017-04

In [2]:
df_ps_time = df_ps_time[df_ps_time['diff_time'] >= pd.Timedelta(0)]
print(df_ps_time.head(50))

                            visit_id process_step       diff_time
447950     1000165_4190026492_760066       step_1 0 days 00:00:03
447949     1000165_4190026492_760066       step_2 0 days 00:00:24
447948     1000165_4190026492_760066       step_3 0 days 00:01:17
447947     1000165_4190026492_760066       step_4 0 days 00:00:37
240557   100019538_17884295066_43909       step_0 0 days 00:00:49
240556   100019538_17884295066_43909       step_0 0 days 00:00:14
240559   100019538_17884295066_43909       step_1 0 days 00:00:23
240558   100019538_17884295066_43909       step_1 0 days 00:00:06
240555   100019538_17884295066_43909       step_1 0 days 00:00:23
240554   100019538_17884295066_43909       step_2 0 days 00:00:47
240553   100019538_17884295066_43909       step_3 0 days 00:01:53
240552   100019538_17884295066_43909       step_4 0 days 00:00:57
352680  100022086_87870757897_149620       step_1 0 days 00:00:22
352679  100022086_87870757897_149620       step_2 0 days 00:00:45
352678  10

In [3]:
# Annahme: Der DataFrame `df` enthält Ihre Daten
df_ps_time['diff_time'] = pd.to_timedelta(df_ps_time['diff_time'])  # Konvertieren Sie die Spalte 'diff_time' in timedelta

# Berechnung der durchschnittlichen Zeit pro process_step
average_time = df_ps_time.groupby('process_step')['diff_time'].mean()

# Ausgabe
print(average_time)

process_step
step_0   0 days 00:03:55.759368778
step_1   0 days 00:01:45.639739788
step_2   0 days 00:01:22.563725836
step_3   0 days 00:02:05.487323076
step_4   0 days 00:02:14.329574416
Name: diff_time, dtype: timedelta64[ns]


In [4]:
df_ps_time['diff_time'] = pd.to_timedelta(df_ps_time['diff_time'])  # Konvertieren der Zeitangaben in timedelta-Objekte

# Durchschnittszeit für jeden process_step berechnen
average_time_per_step = df_ps_time.groupby('process_step')['diff_time'].mean()

# Ausgabe
print(average_time_per_step)

process_step
step_0   0 days 00:03:55.759368778
step_1   0 days 00:01:45.639739788
step_2   0 days 00:01:22.563725836
step_3   0 days 00:02:05.487323076
step_4   0 days 00:02:14.329574416
Name: diff_time, dtype: timedelta64[ns]


In [5]:
# Annahme: Ihr DataFrame heißt `df`
# df_ps_time['diff_time'] = pd.to_timedelta(df_ps_time['diff_time'])  # Konvertierung in timedelta-Objekte

# Gesamtzeit pro process_step summieren
total_time_per_step = df_ps_time.groupby('process_step')['diff_time'].sum()

total_time_per_step

# Häufigkeit der einzelnen process_steps berechnen
count_per_step = df_ps_time['process_step'].value_counts()

# Durchschnitt berechnen
average_time_per_step = total_time_per_step / count_per_step

# Ausgabe
print(average_time_per_step)

process_step
step_0   0 days 00:03:55.759368778
step_1   0 days 00:01:45.639739788
step_2   0 days 00:01:22.563725836
step_3   0 days 00:02:05.487323076
step_4   0 days 00:02:14.329574416
dtype: timedelta64[ns]


In [6]:
total_visits = df_web['visit_id'].nunique()

# Filtern der Datensätze, die einzelnen Schritte
process_s0 = df_web[df_web['process_step'] == 'step_0']['visit_id'].nunique()
process_s1 = df_web[df_web['process_step'] == 'step_1']['visit_id'].nunique()
process_s2 = df_web[df_web['process_step'] == 'step_2']['visit_id'].nunique()
process_s3 = df_web[df_web['process_step'] == 'step_3']['visit_id'].nunique()
process_s4 = df_web[df_web['process_step'] == 'step_4']['visit_id'].nunique()


process_s0, process_s1, process_s2, process_s3, process_s4

# # Berechnung der Completion Rate
# completion_rate = (completed_visits / total_visits) * 100

# # Ausgabe
# print(f"Completion Rate: {completion_rate:.2f}%")

(144902, 119255, 104341, 95093, 89826)

In [7]:
total_visits = df_web['visit_id'].nunique()

# Filtern der Datensätze, die den "confirm"-Schritt erreicht haben
completed_visits = df_web[df_web['process_step'] == 'step_4']['visit_id'].nunique()

#Berechnung der Completion Rate
completion_rate = (completed_visits / total_visits) * 100

# Ausgabe
print(f"Completion Rate: {completion_rate:.2f}%")

Completion Rate: 56.82%


In [8]:
# Gruppieren nach visit_id und process_step, um die Häufigkeit zu ermitteln
step_counts = df_web.groupby(['client_id', 'process_step']).size().reset_index(name='count')

# Identifizieren von visit_id mit fehlerhafter Anzahl (count != 1)
error_visits = step_counts[step_counts['count'] != 1]['client_id'].unique()

# Markieren fehlerhafter visit_id im DataFrame
df_web['error'] = df_web['client_id'].isin(error_visits)

# Ausgabe der Daten mit Fehler-Markierung
print("DataFrame mit Fehlern:")
print(df_web)

# Separater DataFrame mit nur fehlerhaften Besuchern (optional)
error_data = df_web[df_web['error']]

print("\nFehlerhafte Besuche:")
print(error_data)


DataFrame mit Fehlern:
        client_id             visitor_id                      visit_id  \
0         9988021   580560515_7732621733  781255054_21935453173_531117   
1         9988021   580560515_7732621733  781255054_21935453173_531117   
2         9988021   580560515_7732621733  781255054_21935453173_531117   
3         9988021   580560515_7732621733  781255054_21935453173_531117   
4         9988021   580560515_7732621733  781255054_21935453173_531117   
...           ...                    ...                           ...   
755400    9668240   388766751_9038881013   922267647_3096648104_968866   
755401    9668240   388766751_9038881013   922267647_3096648104_968866   
755402    9668240   388766751_9038881013   922267647_3096648104_968866   
755403    9668240   388766751_9038881013   922267647_3096648104_968866   
755404     674799  947159805_81558194550   86152093_47511127657_716022   

       process_step           date_time  error  
0            step_3 2017-04-17 15:27:07

In [9]:
error_data["client_id"].value_counts()

client_id
5574879    111
2274658     90
2128341     84
5951379     81
1595223     81
          ... 
1981525      2
8086343      2
373376       2
1001400      2
9965526      2
Name: count, Length: 69125, dtype: int64

In [10]:
grouped = df_ps_time.groupby("visit_id")

aggregated = grouped.agg({'process_step': 'count'}).reset_index()

sorted = aggregated.sort_values(by="process_step", ascending=True)

sorted

,visit_id,process_step
27911,428277449_21778196135_646079,2
1839,120862933_21819322404_545303,3
32289,478886376_11492186946_609397,3
6081,171886830_38696361268_917175,3
37512,540851138_57035038419_850084,3
...,...,...
50909,694918984_92465751580_342876,62
17765,308874104_3998249411_902973,69
42983,602953935_48759866176_238903,78
73717,961878360_85895454962_607105,79


In [11]:
# Grouping by 'visit_id' and aggregating 'process_step' count
df_web_ps = df_web.groupby("visit_id").agg({'process_step': 'count'}).reset_index()

# Displaying the result
print(df_web_ps)

                            visit_id  process_step
0       100012776_37918976071_457913             2
1          1000165_4190026492_760066             5
2        100019538_17884295066_43909            11
3       100022086_87870757897_149620             5
4       100030127_47967100085_936361             1
...                              ...           ...
158090  999986292_55082159911_561557             2
158091  999988789_76411676596_272843             5
158092  999989069_52203656268_170206             2
158093  999991142_15352368745_399725             1
158094  999992932_41666455053_671149             6

[158095 rows x 2 columns]
